In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/project/archive.zip -d /content/gtsrb

In [11]:
import pandas as pd
import os
from PIL import Image

# Set the base path to your dataset directory
base_path = '/content/gtsrb'

# Assuming CSV annotations are available
# GTSRB typically comes with CSV files that list image names, bounding boxes, and class labels
annotations_csv = pd.read_csv('/content/gtsrb/Train.csv')


In [12]:

# List all directories within the train path (each class directory)
class_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

# Iterate through each class directory and create a 'labels' subdirectory
for class_id in class_dirs:
    labels_path = os.path.join(base_path, class_id, 'labels')
    os.makedirs(labels_path, exist_ok=True)
    print(f"Created or verified labels folder: {labels_path}")


Created or verified labels folder: /content/gtsrb/Test/labels
Created or verified labels folder: /content/gtsrb/Train/labels
Created or verified labels folder: /content/gtsrb/test/labels
Created or verified labels folder: /content/gtsrb/Meta/labels
Created or verified labels folder: /content/gtsrb/meta/labels
Created or verified labels folder: /content/gtsrb/train/labels


In [13]:
for _, row in annotations_csv.iterrows():
    # Construct the image path
    img_path = os.path.join(base_path, row['Path'])
    image = Image.open(img_path)
    iw, ih = image.size

    # Normalize the bounding box dimensions
    x_center = (row['Roi.X1'] + (row['Roi.X2'] - row['Roi.X1']) / 2) / iw
    y_center = (row['Roi.Y1'] + (row['Roi.Y2'] - row['Roi.Y1']) / 2) / ih
    width = (row['Roi.X2'] - row['Roi.X1']) / iw
    height = (row['Roi.Y2'] - row['Roi.Y1']) / ih

    # Format for YOLO
    yolo_data = f"{row['ClassId']} {x_center} {y_center} {width} {height}\n"

    # Write to the corresponding .txt file
    txt_path = os.path.join(base_path, 'train/labels', os.path.splitext(os.path.basename(row['Path']))[0] + '.txt')
    with open(txt_path, 'a') as file:
        file.write(yolo_data)
